In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [2]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet_FC'
# NETWORK = 'Resnet'
NETWORK = 'Resnet_res2c'
# NETWORK = 'Resnet_res3d'
# NETWORK = 'Resnet_res4f'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
# NETWORK = 'EfficientNetB7'
# NETWORK = 'EfficientNetV2M'
# NETWORK = 'NASNetLarge'
# NETWORK = 'ResNet152V2'
# NETWORK = 'ConvNeXtBase'

METHOD = "Max-Mean"

In [3]:
WORKING_DIRECTORY = '/home/amirhosein/HECKTOR2022/WHOLEIMAGE_MAMIP'

PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE_{METHOD}.csv"
# Features_DenseNet201_MA-MIP_WHOLE-IMAGE_Max-Mean

features_path = os.path.join(WORKING_DIRECTORY, f"ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"ProcessedFeatures_{METHOD}")

processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE_{METHOD}.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/home/amirhosein/HECKTOR2022/hecktor2022_patient_endpoint_training.csv"

In [4]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
0,CHUM-001,4.662766,4.508459,3.635648,4.601479,6.615333,3.004359,3.899298,9.431976,4.713881,...,5.796395,4.444568,6.637720,3.939236,3.863583,2.942393,4.449009,5.112864,2.457382,5.215610
1,CHUM-002,5.517929,4.510674,3.421755,4.709925,7.019833,2.821013,3.764581,9.417235,5.576050,...,5.789610,3.206661,6.608598,4.237351,3.694597,3.028637,4.366594,4.756105,2.457555,5.076906
2,CHUM-006,4.858120,4.397993,4.129549,4.809740,6.847592,3.041794,3.927901,9.349664,7.281087,...,5.809862,4.860772,6.560032,4.011434,4.505952,2.741225,5.683290,5.408144,2.451912,5.186686
3,CHUM-007,3.564949,4.415835,3.912434,4.763706,6.871284,3.193196,3.915397,9.437322,5.524411,...,5.795392,5.357600,6.604254,4.057689,4.435986,2.680756,5.766419,4.877578,2.456779,5.044932
4,CHUM-008,3.094451,4.445643,3.939131,4.813448,6.413505,3.152784,3.872820,9.416096,5.413524,...,5.807824,5.205809,6.591450,3.855028,4.043146,2.663939,5.178743,4.589118,2.455964,5.067176


# Up to here

Read Outcome csv

In [5]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [6]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [7]:
OD.columns[0]

'Patient_ID'

In [8]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [9]:
OD.columns[0]


'Patient_ID'

In [10]:
PET_outcome_data.columns[0]

'Patient_ID'

In [11]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
0,CHUM-001,4.662766,4.508459,3.635648,4.601479,6.615333,3.004359,3.899298,9.431976,4.713881,...,5.796395,4.444568,6.637720,3.939236,3.863583,2.942393,4.449009,5.112864,2.457382,5.215610
1,CHUM-002,5.517929,4.510674,3.421755,4.709925,7.019833,2.821013,3.764581,9.417235,5.576050,...,5.789610,3.206661,6.608598,4.237351,3.694597,3.028637,4.366594,4.756105,2.457555,5.076906
2,CHUM-006,4.858120,4.397993,4.129549,4.809740,6.847592,3.041794,3.927901,9.349664,7.281087,...,5.809862,4.860772,6.560032,4.011434,4.505952,2.741225,5.683290,5.408144,2.451912,5.186686
3,CHUM-007,3.564949,4.415835,3.912434,4.763706,6.871284,3.193196,3.915397,9.437322,5.524411,...,5.795392,5.357600,6.604254,4.057689,4.435986,2.680756,5.766419,4.877578,2.456779,5.044932
4,CHUM-008,3.094451,4.445643,3.939131,4.813448,6.413505,3.152784,3.872820,9.416096,5.413524,...,5.807824,5.205809,6.591450,3.855028,4.043146,2.663939,5.178743,4.589118,2.455964,5.067176


In [12]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [13]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [14]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,248,249,250,251,252,253,254,255,Relapse,RFS
0,CHUM-001,4.662766,4.508459,3.635648,4.601479,6.615333,3.004359,3.899298,9.431976,4.713881,...,6.637720,3.939236,3.863583,2.942393,4.449009,5.112864,2.457382,5.215610,0,1704
1,CHUM-002,5.517929,4.510674,3.421755,4.709925,7.019833,2.821013,3.764581,9.417235,5.576050,...,6.608598,4.237351,3.694597,3.028637,4.366594,4.756105,2.457555,5.076906,1,439
2,CHUM-006,4.858120,4.397993,4.129549,4.809740,6.847592,3.041794,3.927901,9.349664,7.281087,...,6.560032,4.011434,4.505952,2.741225,5.683290,5.408144,2.451912,5.186686,0,1186
3,CHUM-007,3.564949,4.415835,3.912434,4.763706,6.871284,3.193196,3.915397,9.437322,5.524411,...,6.604254,4.057689,4.435986,2.680756,5.766419,4.877578,2.456779,5.044932,0,1702
4,CHUM-008,3.094451,4.445643,3.939131,4.813448,6.413505,3.152784,3.872820,9.416096,5.413524,...,6.591450,3.855028,4.043146,2.663939,5.178743,4.589118,2.455964,5.067176,0,1499


In [15]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)